<a href="https://colab.research.google.com/github/MamMates/ml-food-recommendation/blob/%233-recommender-model/MamMates_Food_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow-recommenders

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs

In [ ]:
dataset_link = "https://docs.google.com/spreadsheets/d/1o0f663wcmMfta_PAILOJzkvn09_Mbjw5zBxgcDk1Au0"

df_dataset = pd.read_csv(f'{dataset_link}/export?gid=0&format=csv')
df_dataset

,id_user,id_food,rating
0,63,15,1
1,66,20,2
2,37,1,2
3,39,13,3
4,52,18,1
...,...,...,...
995,35,17,3
996,79,20,1
997,92,1,1
998,1,14,3


In [ ]:
df_food_info = pd.read_csv(f'{dataset_link}/export?gid=1905501804&format=csv')
df_food_info.head()

,id_food,food_name
0,1,Donat Ubi Mawar
1,2,Donat Ubi Mawar
2,3,Kue Cubit Maniez
3,4,Kue Cubit Maniez
4,5,Kue Lapis Legit


In [ ]:
len(df_dataset.id_user.unique())

100

In [ ]:
len(df_dataset.id_food.unique())

20

In [ ]:
ratings = tf.data.Dataset.from_tensor_slices(
    {"user_id": df_dataset.id_user.astype(str),
     "food_id": df_dataset.id_food.astype(str)}
)

In [ ]:
for x in ratings.take(2).as_numpy_iterator():
  print(x)

{'user_id': b'63', 'food_id': b'15'}
{'user_id': b'66', 'food_id': b'20'}


In [ ]:
foods = tf.data.Dataset.from_tensor_slices(
    df_food_info.id_food.astype(str)
)

In [ ]:
for x in foods.take(2).as_numpy_iterator():
  print(x)

b'1'
b'2'


In [ ]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(1000, seed=42, reshuffle_each_iteration=False)
train = shuffled.take(800)
test = shuffled.skip(800).take(200)

In [ ]:
food_ids = foods.batch(32)
user_ids = ratings.batch(32).map(lambda x: x["user_id"])

unique_food_ids = np.unique(np.concatenate(list(food_ids)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_food_ids[:10]

array([b'1', b'10', b'11', b'12', b'13', b'14', b'15', b'16', b'17',
       b'18'], dtype=object)

In [ ]:
embedding_dimension = 32

In [ ]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [ ]:
food_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_food_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_food_ids) + 1, embedding_dimension)
])

In [ ]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=foods.batch(32).map(food_model)
)

In [ ]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [ ]:
from typing import Dict, Text

class MamMatesModel(tfrs.Model):

  def __init__(self, user_model, food_model):
    super().__init__()
    self.food_model: tf.keras.Model = food_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    user_embeddings = self.user_model(features["user_id"])
    positive_food_embeddings = self.food_model(features["food_id"])

    return self.task(user_embeddings, positive_food_embeddings)

In [ ]:
class NoBaseClassMammatesModel(tf.keras.Model):

  def __init__(self, user_model, food_model):
    super().__init__()
    self.food_model: tf.keras.Model = food_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    with tf.GradientTape() as tape:

      user_embeddings = self.user_model(features["user_id"])
      positive_food_embeddings = self.food_model(features["food_id"])
      loss = self.task(user_embeddings, positive_food_embeddings)

      regularization_loss = sum(self.losses)

      total_loss = loss + regularization_loss

    gradients = tape.gradient(total_loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

  def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    user_embeddings = self.user_model(features["user_id"])
    positive_food_embeddings = self.food_model(features["food_id"])
    loss = self.task(user_embeddings, positive_food_embeddings)

    regularization_loss = sum(self.losses)

    total_loss = loss + regularization_loss

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

In [ ]:
model = MamMatesModel(user_model, food_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
cached_train = train.shuffle(1000).batch(32).cache()
cached_test = test.batch(32).cache()

In [ ]:
model.fit(cached_train, epochs=3)

Epoch 1/3
25/25 [==============================] - 4s 49ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.2025 - factorized_top_k/top_10_categorical_accuracy: 0.4787 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 110.9281 - regularization_loss: 0.0000e+00 - total_loss: 110.9281
Epoch 2/3
25/25 [==============================] - 3s 102ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0012 - factorized_top_k/top_5_categorical_accuracy: 0.3237 - factorized_top_k/top_10_categorical_accuracy: 0.6488 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 109.5470 - regularization_loss: 0.0000e+00 - total_loss: 109.5470
Epoch 3/3
25/25 [==============================] - 4s 149ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0088 - factorized_top_k/top_5_categorical_accuracy: 0.413

In [ ]:
model.evaluate(cached_test, return_dict=True)

7/7 [==============================] - 1s 94ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0250 - factorized_top_k/top_5_categorical_accuracy: 0.2400 - factorized_top_k/top_10_categorical_accuracy: 0.4800 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 90.3494 - regularization_loss: 0.0000e+00 - total_loss: 90.3494


{'factorized_top_k/top_1_categorical_accuracy': 0.02500000037252903,
 'factorized_top_k/top_5_categorical_accuracy': 0.23999999463558197,
 'factorized_top_k/top_10_categorical_accuracy': 0.47999998927116394,
 'factorized_top_k/top_50_categorical_accuracy': 1.0,
 'factorized_top_k/top_100_categorical_accuracy': 1.0,
 'loss': 16.14521026611328,
 'regularization_loss': 0,
 'total_loss': 16.14521026611328}

In [ ]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
  tf.data.Dataset.zip((foods.batch(32), foods.batch(32).map(model.food_model)))
)

_, titles = index(tf.constant(["14"]))
print(f"Recommendations for user with ID 14: {titles[0, :3]}")

Recommendations for user with ID 14: [b'13' b'14' b'12']


In [ ]:
list_titles = titles.numpy().astype(int).tolist()
list_titles[0]

[13, 14, 12, 2, 18, 20, 11, 10, 7, 9]

In [ ]:
filtered_foods = df_food_info[df_food_info['id_food'].isin(list_titles[0])]

id_to_food = dict(zip(filtered_foods['id_food'], filtered_foods['food_name']))

food_names = [id_to_food.get(id) for id in list_titles[0]]
print(f"Recommendations for user with ID 14: {food_names[:3]}")

Recommendations for user with ID 14: ['Roti Bakar Cokelat Keju', 'Roti Bakar Niqmat', 'Donat Ubi Rasa Cinta']


In [ ]:
import tempfile
import os

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.saved_model.save(index, export_path)

export_path = /tmp/1



In [ ]:
!zip -r model.zip /tmp/1/

updating: tmp/1/ (stored 0%)
updating: tmp/1/fingerprint.pb (stored 0%)
updating: tmp/1/variables/ (stored 0%)
updating: tmp/1/variables/variables.index (deflated 34%)
updating: tmp/1/variables/variables.data-00000-of-00001 (deflated 15%)
updating: tmp/1/assets/ (stored 0%)
updating: tmp/1/saved_model.pb (deflated 84%)
